<a href="https://colab.research.google.com/github/pam-lab/JupyterFiles/blob/main/NLP_ParsBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
# https://drive.google.com/file/d/18l4Ca8KBkCo0-Eelt6dxAo6ve0lGEGuJ/view?usp=sharing
!gdown 18l4Ca8KBkCo0-Eelt6dxAo6ve0lGEGuJ

Downloading...
From: https://drive.google.com/uc?id=18l4Ca8KBkCo0-Eelt6dxAo6ve0lGEGuJ
To: /content/dataset_annotated_sentiment.json
100% 1.74M/1.74M [00:00<00:00, 148MB/s]


In [4]:
import json
from pathlib import Path
dataset = json.loads(Path('dataset_annotated_sentiment.json').read_text())

In [5]:
pip install hazm -Uq

In [6]:
from hazm import Normalizer

normalizer = Normalizer()

In [7]:
def read_data_convert(data,voting):
    """
    this function gets a split and returns its text and labels
    """

    # this is correct. But it shows bad because of RTL
    labels_dict = {'خنثی': 0, 'مثبت': 1, 'منفی': -1}
    text_array = []
    label_array = []
    if voting:
        for item in data:
            label_item = item['annotations']
            pos = label_item.count('مثبت')
            neg = label_item.count('منفی')
            neutral = label_item.count('خنثی')
            if pos == neg == neutral:
                continue
            text_array.append(normalizer.normalize(item['text']))
            number_label = "مثبت" if pos > 1 else ("منفی" if neg > 1 else "خنثی")
            label_array.append(number_label)
    else:
        for item in data:
            for label in item['annotations']:
                if isinstance(label, str):  
                    text_array.append( normalizer.normalize(item['text']))
                    label_array.append(label)

    return text_array, label_array

In [8]:
pip install transformers -Uq

In [9]:
from transformers import pipeline
from transformers import BertTokenizer, AutoModelForSequenceClassification

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = BertTokenizer.from_pretrained(model_name_or_path)

model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b

In [10]:
#!pip install ipdb -Uqq
#%pdb on

In [ ]:
train_texts, train_labels = read_data_convert(dataset['train'],voting=False)
val_texts, val_labels = read_data_convert(dataset['eval'],voting=True)
test_texts, test_labels = read_data_convert(dataset['test'],voting=True)

train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True,max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True,max_length=256)

In [ ]:
import pandas as pd
train_labels_df=pd.get_dummies(pd.DataFrame(train_labels))
val_labels_df = pd.get_dummies(pd.DataFrame(val_labels))
test_labels_df = pd.get_dummies(pd.DataFrame(test_labels))
train_labels_df

In [ ]:
train_labels_df.value_counts()

In [ ]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels_df.values)
val_dataset = SentimentDataset(val_encodings, val_labels_df.values)
test_dataset = SentimentDataset(test_encodings, test_labels_df.values)

In [1]:
train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)

NameError: ignored

In [ ]:
batch = next(iter(train_loader))
model(attention_mask=batch['attention_mask'],input_ids=batch['input_ids'])

In [63]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


ValueError: ignored